<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/LING5412_Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tarfile
import pandas as pd
import csv
import seaborn as sns

# Loading data

In [2]:

# Unzip the dataset
!unzip "/content/dontpatronizeme_v1.3.zip" -d "/content/drive/MyDrive/"

Archive:  /content/dontpatronizeme_v1.3.zip
   creating: /content/drive/MyDrive/dontpatronizeme_v1.3/
  inflating: /content/drive/MyDrive/dontpatronizeme_v1.3/dontpatronizeme_categories.tsv  
  inflating: /content/drive/MyDrive/dontpatronizeme_v1.3/README.txt  
  inflating: /content/drive/MyDrive/dontpatronizeme_v1.3/dontpatronizeme_pcl.tsv  


In [4]:
file = open(r'/content/drive/MyDrive/dontpatronizeme_v1.3/dontpatronizeme_pcl.tsv')
reader = csv.reader(file, delimiter="\t")
data = []
for row in reader:
  data.append(row)


In [5]:
df = pd.DataFrame(data[5:],  columns = ['docID', 'keyword', 'country', 'paragraph', 'label' ] )
df

,docID,keyword,country,paragraph,label
0,@@4703096,immigrant,jm,NBC and Spanish-language Univision both declin...,0
1,@@25567226,in-need,hk,A second T-Home project is being launched in t...,0
2,@@1824078,poor-families,tz,Camfed would like to see this trend reversed ....,4
3,@@1921089,refugee,tz,Kagunga village was reported to lack necessary...,0
4,@@40039380,women,ng,Haruna stressed the need for specific approach...,0
...,...,...,...,...,...
10053,@@16413808,immigrant,my,"To me , I am always mindful that we are dealin...",4
10054,@@8676630,vulnerable,jm,Other themes included promoting the inclusion ...,0
10055,@@7688552,immigrant,gb,It came as the CDU was also humiliated by the ...,0
10056,@@4916290,hopeless,in,"Those were only days of helplessness , she say...",0


# Exploring data

In [6]:
# Length of text
def length (txt):
  length = len(txt.split())
  return length

txt_length = df['paragraph'].apply(lambda x: length(x))
txt_length.sort_values(ascending = False)

3534    1519
6266    1095
8519    1040
4613     772
8819     729
        ... 
1930       4
1385       3
7375       3
5112       3
5742       0
Name: paragraph, Length: 10058, dtype: int64

In [7]:
# Observing labels
df['label'].value_counts()

0    8206
1     906
3     435
4     369
2     142
Name: label, dtype: int64

In [8]:
# Turning labels to binary

label_dic = {0:0,
             1:0,
             2:1,
             3:1,
             4:1}
df['label'] = df['label'].map(label_dic)
print(df['label'].value_counts())
sns.countplot(x='label', data=df)
plt.show()

Series([], Name: label, dtype: int64)


ValueError: ignored

# Developing models

# w